In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance

import trecs
from trecs.models import ImplicitMF, ImplicitMFLFD
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, AverageFeatureScoreRange

In [2]:
mf = ImplicitMF(num_users=200, num_items=50, num_latent_factors=20)
mf.add_metrics(MSEMeasurement())
mf.startup_and_train(20)

100%|██████████| 20/20 [00:00<00:00, 584.67it/s]


In [3]:
mflfd = ImplicitMFLFD(num_users=200, num_items=50, num_latent_factors=20)
mflfd.add_metrics(MSEMeasurement())

In [4]:
mflfd.startup_and_train(20)
mflfd.run(10)

100%|██████████| 10/10 [00:00<00:00, 234.43it/s]


In [5]:
mflfd.rec

array([[43, 14,  3, ..., 17, 13, 44],
       [13,  2, 25, ...,  4,  0,  9],
       [39, 13, 42, ..., 26, 15,  5],
       ...,
       [ 4,  7, 25, ..., 23, 45, 46],
       [13, 22, 44, ..., 35,  9, 15],
       [18, 13, 35, ..., 22, 47, 17]])

In [6]:
mflfd.item_indices[2]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [7]:
mflfd.predicted_scores

PredictedScores([[-0.06273281,  0.06886859, -0.00567519, ...,
                   0.05445335,  0.02260769,  0.05934655],
                 [ 0.12988215, -0.0346267 ,  0.24468186, ...,
                   0.10893789,  0.05290764,  0.00351817],
                 [ 0.04296566,  0.16896895,  0.06580345, ...,
                   0.05888834, -0.04893276,  0.09713374],
                 ...,
                 [ 0.25171803, -0.01905507,  0.04780995, ...,
                   0.03291001,  0.0211827 ,  0.24986726],
                 [-0.05419834,  0.2483475 ,  0.03768484, ...,
                   0.06427463, -0.00132529, -0.06175335],
                 [ 0.18482815,  0.19309881,  0.17933266, ...,
                   0.19003587, -0.01764408,  0.07171722]])

In [9]:
top_n_limit=25
k=10

row = np.repeat(mflfd.users.user_vector, mflfd.item_indices.shape[1])
row = row.reshape((mflfd.num_users, -1))
s_filtered = mflfd.predicted_scores[row, mflfd.item_indices]

negated_scores = -1 * s_filtered  # negate scores so indices go from highest to lowest
# break ties using a random score component
scores_tiebreak = np.zeros(
    negated_scores.shape, dtype=[("score", "f8"), ("random", "f8")]
)
scores_tiebreak["score"] = negated_scores
scores_tiebreak["random"] = mflfd.random_state.random(negated_scores.shape)
top_k = scores_tiebreak.argpartition(top_n_limit - 1, order=["score", "random"])[:, :top_n_limit]
# now we sort within the top k
row = np.repeat(mflfd.users.user_vector, top_n_limit).reshape((mflfd.num_users, -1))
# again, indices should go from highest to lowest
sort_top_k = scores_tiebreak[row, top_k].argsort(order=["score", "random"])
top_k_recs = mflfd.item_indices[row, top_k[row, sort_top_k]]

#dims are attribute, items, users
top_k_att = mflfd.items_hat[:, top_k_recs[:]].swapaxes(1,2)


In [10]:
#test=s_filtered[row, top_k_recs]
#top_k_recs[user, :]

NameError: name 'user' is not defined

In [11]:
mflfd.items_hat.shape

(20, 50)

In [15]:
print(top_k_recs.shape)
print(top_k_att.shape)
#items_att_filt.swapaxes(1,2)
#items_att_filt.shape

(200, 25)
(20, 25, 200)


In [13]:
top_item_att = items_att_filt[:,0, :]
top_item_idx = top_k_recs[:, 0].reshape(top_k_recs.shape[0],1)

NameError: name 'items_att_filt' is not defined

In [ ]:
print(top_item_att.shape, top_item_idx.shape)

In [ ]:
mflfd.items_hat[0][]

In [ ]:
# all_recs_idx = np.empty([mflfd.users_hat.shape[0], k])
# print(all_recs_idx.shape)
# print(top_item_idx.shape)
# all_recs[:, 0] = top_item_idx[:,None]
all_recs_idx = top_k_recs[:,0].reshape(top_k_recs.shape[0],1)
top_k_recs[0,0]

In [88]:
#store the id of the highest predicted item for each user as a start
all_recs_idx = top_k_recs[:,0].reshape(top_k_recs.shape[0],1)
for idx, user in enumerate(mflfd.users_hat):

        #get the top rec and add that as the first item for each user
        user_item_feats = top_k_att[:,:,idx]
        #user_item_feats_idx = [0]
        user_max_idx = top_k_recs[idx, 0] 
        recs_idxs = [user_max_idx]
        
        #hold the features of the recommended items
        recs_features = mflfd.items_hat[:,user_max_idx]
        
        distances = []
        
        for rec in range(1,k):
            
            #user_item_feats_idx.append(rec)
            
            #drop element from user_item_feats that has been put in the recommendations
            ##DONT DO THIS BECAUSE IT MAKES INDEXING MORE COMPLEX
            #mask = np.invert(np.isin(user_item_feats, recs_features))
            #user_item_feats = user_item_feats[mask].reshape(user_item_feats.shape[0], -1).shape
            
            if rec == 1:
                #for the second item, just use the first item values
                centroid = recs_features
            else:
                centroid = np.nanmean(recs_features, axis=0)

            centroid = centroid.reshape(1, -1)

            #set all the previously chosen item features to the centroid, so they will not be selected again
            #don't want to just remove rows because it will throw of the indexing
            user_item_feats[:, 0:rec]=centroid.T

            d = pairwise_distances(X=centroid, Y=user_item_feats.T, metric='cityblock',force_all_finite='allow_nan' )
            most_distant = np.argmax(d)
            
            if rec == 1:
                print (idx, d.max())
            
            distances.append(d.max())
            
            most_distant_feats = user_item_feats.T[most_distant]
            
            #get the index of the most distant item in the top k recs
            recs_idxs.append(top_k_recs[idx, most_distant])
            #recs_idxs.append(most_distant)
            #get the item index from the original array of indices, not the constrained array
            #orig_recs_idxs.append(top_k_recs[idx, most_distant])
            #recs_preds.append(n_ratings[idx][most_distant])

            recs_features = np.vstack((recs_features, user_item_feats[:, most_distant]))


0 3.402183293702843
1 3.3282836051426794
2 3.1810777321950052
3 3.2615364374606375
4 3.563153682192554
5 3.433947842729087
6 3.160564392584609
7 3.5120364831365594
8 3.404685849218776
9 3.2920821833523712
10 3.163005498457879
11 3.385218586302316
12 3.047411339230747
13 3.4532046083827987
14 3.536616201976738
15 3.5295930587276168
16 3.1552650262606523
17 3.5256761975658857
18 3.157954548118826
19 3.257019890758991
20 3.2637878661112043
21 3.31269889193703
22 3.536616201976738
23 3.182288330934464
24 3.6876358221879193
25 3.602472721277783
26 3.481206120359996
27 3.2507231578679825
28 3.6876358221879193
29 3.2920821833523712
30 3.2615364374606375
31 3.5989635831464617
32 3.0840593432707206
33 3.5198374123790015
34 3.4426368355488917
35 3.2972170606539124
36 3.5595993041825684
37 3.3788825139322363
38 3.183177774830137
39 3.2708742968846924
40 3.247427705678935
41 3.230221166757755
42 3.7070597473423446
43 3.602472721277783
44 3.247427705678935
45 3.2920821833523712
46 3.525676197565885

In [104]:
top_k_att.shape
#last_user_att = top_k_att[:,:,199]
#print(recs_idxs)
#print(distances)
last_user_att.shape

(20, 25)

In [105]:
#distance.cityblock(mflfd.items_hat[18], mflfd.items_hat[9])

all_ds = pairwise_distances(X=last_user_att[18], Y=last_user_att, metric='cityblock',force_all_finite='allow_nan' )

ValueError: Expected 2D array, got 1D array instead:
array=[-0.04206372 -0.04206372 -0.04206372 -0.04206372 -0.04206372 -0.04206372
 -0.04206372 -0.04206372 -0.04206372 -0.0477606   0.00414546 -0.03169692
  0.05318124 -0.05393281 -0.06029785 -0.04459911 -0.16788761 -0.08735656
  0.01095976 -0.07153796  0.00529734 -0.01609356 -0.03375366  0.15719939
  0.03590416].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [100]:
all_ds[18].max()
#top_k_recs
#user_max_idx = top_k_recs[idx, 0] 

4.148073029217128

In [54]:
np.isin(user_item_feats, recs_features)
mask = np.invert(np.isin(user_item_feats, recs_features))
#mask.shape
#result = user_item_feats[mask,...]

#drop element from user_item_feats that has been put in the recommendations
user_item_feats = user_item_feats[mask].reshape(user_item_feats.shape[0], -1).shape

(20, 24)

In [52]:
user_item_feats.shape
#C = np.delete(C, 1, 1)  # delete second column of C
#np.where(np.isin(user_item_feats, recs_features), user_item_feats)

(20, 25)

In [25]:
mask = np.ones(shape=user_item_feats.shape, dtype=bool)
mask[recs_idxs] = False

#mask[[0,2,4]] = False

IndexError: index 43 is out of bounds for axis 0 with size 20

In [ ]:
def latent_factors_diversification(user_features, item_features, n_recs=10, top_n_limit=None):


    hat_ratings = np.dot(user_features, item_features.T) 

    if top_n_limit:
        #if constraining by top n, only retain the top n ratings within each user
        ind=np.argpartition(hat_ratings,-top_n_limit)[:,-top_n_limit:]
        n_ratings = np.take(hat_ratings, ind)
    else:
        #if not constraining by top n, retail all item indices for all users. 
        #If this is the case, in all_user_recs, recs_idxs should match original_recs_idxs
        ind=np.tile(np.arange(0,len(item_features)),(len(user_features),1))
        n_ratings = hat_ratings



    all_user_recs = dict()
    
    max_idx = np.argmax(n_ratings, axis=1)
    top_items=item_features[max_idx]
    
    all_recs = np.empty([user_features.shape[0],item_features.shape[1], n_recs])
    #all_recs = None
    

    for idx, user in enumerate(user_features):

        user_item_feats = item_features[ind[idx]]
        user_max_idx = np.argmax(n_ratings[idx])

        #get the top rec and add that as the first item for each user
        user_max = max_idx[idx]
        recs_features = top_items[idx]
        recs_idxs = [max_idx[idx]]
        recs_preds = [n_ratings[idx][user_max]]
        orig_recs_idxs = [ind[idx, user_max]]



        for rec in range(1,n_recs):
            if rec == 1:
                #for the second item, just use the first item values
                centroid = recs_features
            else:
                centroid = np.nanmean(recs_features, axis=0)

            centroid = centroid.reshape(1, -1)

            #set all the previously chosen item features to the centroid, so they will not be selected again
            #don't want to just remove rows because it will throw of the indexing
            user_item_feats[recs_idxs]=centroid

            d = pairwise_distances(X=centroid, Y=user_item_feats, metric='cityblock',force_all_finite='allow_nan' )
            most_distant = np.argmax(d)

            recs_idxs.append(most_distant)
            #get the item index from the original array of indices, not the constrained array
            orig_recs_idxs.append(ind[idx, most_distant])
            recs_preds.append(n_ratings[idx][most_distant])

            recs_features = np.vstack((recs_features, user_item_feats[most_distant]))

        all_recs[idx, :, :]=recs_features
            
        all_user_recs[idx]={'user_feats': user,
                        'original_recs_idx':orig_recs_idxs,
                        'recs_idx':recs_idxs,
                        'recs_features':recs_features,
                        'recs_preds':recs_preds}

        
    return all_recs, all_user_recs

In [ ]:
n_ratings.shape

In [ ]:
import datetime

x = datetime.datetime.now()
print(x) 